# Installing
----

In [ ]:
!apt-get install flex

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libfl-dev libfl2 libsigsegv2 m4
Suggested packages:
  bison flex-doc m4-doc
The following NEW packages will be installed:
  flex libfl-dev libfl2 libsigsegv2 m4
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 545 kB of archives.
After this operation, 1,511 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsigsegv2 amd64 2.12-1 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 m4 amd64 1.4.18-1 [197 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 flex amd64 2.6.4-6 [316 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfl2 amd64 2.6.4-6 [11.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu

In [ ]:
!apt-get install bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libbison-dev
Suggested packages:
  bison-doc
The following NEW packages will be installed:
  bison libbison-dev
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 605 kB of archives.
After this operation, 1,811 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libbison-dev amd64 2:3.0.4.dfsg-1build1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 bison amd64 2:3.0.4.dfsg-1build1 [266 kB]
Fetched 605 kB in 1s (978 kB/s)
Selecting previously unselected package libbison-dev:amd64.
(Reading database ... 160863 files and directories currently installed.)
Preparing to unpack .../libbison-dev_2%3a3.0.4.dfsg-1build1_amd64.

# Sample lex
----

In [ ]:
!wget http://tug.ctan.org/graphics/sketch/y.tab.h
!wget http://tug.ctan.org/graphics/sketch/y.tab.c
# !wget http://tug.ctan.org/graphics/sketch/lex.yy.c

--2021-05-25 14:55:12--  http://tug.ctan.org/graphics/sketch/y.tab.h
Resolving tug.ctan.org (tug.ctan.org)... 155.101.98.136
Connecting to tug.ctan.org (tug.ctan.org)|155.101.98.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3565 (3.5K) [text/plain]
Saving to: ‘y.tab.h’

y.tab.h             100%[===================>]   3.48K  --.-KB/s    in 0s      

2021-05-25 14:55:13 (462 MB/s) - ‘y.tab.h’ saved [3565/3565]

--2021-05-25 14:55:13--  http://tug.ctan.org/graphics/sketch/y.tab.c
Resolving tug.ctan.org (tug.ctan.org)... 155.101.98.136
Connecting to tug.ctan.org (tug.ctan.org)|155.101.98.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74631 (73K) [text/plain]
Saving to: ‘y.tab.c’

y.tab.c             100%[===================>]  72.88K  --.-KB/s    in 0.1s    

2021-05-25 14:55:13 (575 KB/s) - ‘y.tab.c’ saved [74631/74631]



In [ ]:
%%writefile sample.l
%{
int COMMENT=0;
%}
identifier [a-zA-Z][a-zA-Z0-9]*
%%
#.* {printf("\n%s is a preprocessor directive",yytext);}
int |
float |
char |
double |
while |
for |
struct |
typedef |
do |
if |
break |
continue |
void |
switch |
return |
else |
goto {printf("\n\t%s is a keyword",yytext);}
"/*" {COMMENT=1;}{printf("\n\t %s is a COMMENT",yytext);}
{identifier}\( {if(!COMMENT)printf("\nFUNCTION \n\t%s",yytext);}
\{  {if(!COMMENT)printf("\n BLOCK BEGINS");}
\}  {if(!COMMENT)printf("BLOCK ENDS ");}
{identifier}(\[[0-9]*\])? {if(!COMMENT) printf("\n %s IDENTIFIER",yytext);}
\".*\" {if(!COMMENT)printf("\n\t %s is a STRING",yytext);}
[0-9]+ {if(!COMMENT) printf("\n %s is a NUMBER ",yytext);}
\)(\:)? {if(!COMMENT)printf("\n\t");ECHO;printf("\n");}
\( ECHO;
= {if(!COMMENT)printf("\n\t %s is an ASSIGNMENT OPERATOR",yytext);}
\<= |
\>= |
\< |
== |
\> {if(!COMMENT) printf("\n\t%s is a RELATIONAL OPERATOR",yytext);}
%%
int main(int argc, char **argv)
{
FILE *file;
file=fopen("sample.c","r");
if(!file)
{
printf("could not open the file");
exit(0);
}
yyin=file;
yylex();
printf("\n");
return(0);
}
int yywrap()
{
return(1);
}


Writing sample.l


In [ ]:
%%writefile sample.c
#include<stdio.h>
void main()
{
int a,b,c;
a=1;
b=2;
c=a+b;
printf("Sum:%d",c);
}

Writing sample.c


In [ ]:
%%shell
lex sample.l
gcc -o sample lex.yy.c
./sample


#include<stdio.h> is a preprocessor directive

	void is a keyword 
FUNCTION 
	main(
	)


 BLOCK BEGINS

	int is a keyword 
 a IDENTIFIER,
 b IDENTIFIER,
 c IDENTIFIER;

 a IDENTIFIER
	 = is an ASSIGNMENT OPERATOR
 1 is a NUMBER ;

 b IDENTIFIER
	 = is an ASSIGNMENT OPERATOR
 2 is a NUMBER ;

 c IDENTIFIER
	 = is an ASSIGNMENT OPERATOR
 a IDENTIFIER+
 b IDENTIFIER;

FUNCTION 
	printf(
	 "Sum:%d" is a STRING,
 c IDENTIFIER
	)
;
BLOCK ENDS 


# Practical 1
Implement a Lexical Analyzer using LEX for a subset of C. Cross check your output with Stanford LEX.
----

In [ ]:
%%writefile practical1.l
%{

FILE* yyin;
%}
DIGIT [0-9]
NUMBER {DIGIT}+
REAL {DIGIT}*[.]{NUMBER}
TEXT [A-Za-z]
KEYWORDS "auto"|"static"
DATATYPE "int"|"float"|"double"|"long"|"void"|"char"
CONDITIONAL "if"|"else"|"else if"|"switch"|"case"
ITERATIVE "for"|"while"|"do"
PREPROCESSOR "#include"[^\n]*";"
SC ";"
IDENTIFIER {TEXT}({DIGIT}|{TEXT}|"_")*
NONIDENTIFIER {DIGIT}({TEXT}|{DIGIT}|"_")*
ARITH_OP "+"|"-"|"/"|"%"|"*";
LOGICAL_OP "&&"|"||"|"!"|"!="
REL_OP "<"|">"|"<="|">="|"=="
UNARY "++"|"--"


%%
[ \n\t]+ ;
"#include<stdio.h>" {printf("%s\t==> PREPROCESSOR\n",yytext);}
{CONDITIONAL} { printf("%s\t==> CONDITIONAL\n",yytext);}
{ITERATIVE} {printf("%s\t==> ITERATIVE CONSTRUCT\n",yytext);}

{DATATYPE}  {printf("%s\t==> DATATYPE\n",yytext);}
{KEYWORDS} {printf("%s\t==> KEYWORDS\n",yytext);}
{IDENTIFIER} {printf("%s\t==> IDENTIFIER\n",yytext);}
{REAL} {printf("%s\t==> REAL CONSTANT\n",yytext);}
{NUMBER} {printf("%s\t==> CONSTAINT INTEGER\n",yytext);}
{NONIDENTIFIER} {printf("%s\t==> NONIDENTIFIER\n",yytext);}
{SC} {printf("%s\t==> DELIMITER\n",yytext);}
{UNARY} {printf("%s\t==> UNARY OP\n",yytext);}
{ARITH_OP} {printf("%s\t==> ARITHMETIC OPERATOR\n",yytext);}
{LOGICAL_OP} {printf("%s\t==> LOGICAL OP\n",yytext);}
{REL_OP} {printf("%s\t==> RELATIONAL OP\n",yytext);}
"=" {printf("%s\t==> ASSIGNMENT OP\n",yytext);}
"{"  {printf("%s\t==> BLOCK BEGIN\n",yytext);}
"}"  {printf("%s\t==> BLOCK END\n",yytext);}
"("  {printf("%s\t==> PARANTHESIS BEGIN\n",yytext);}
")"  {printf("%s\t==> PARENTHESIS END\n",yytext);}
. ;


%%
int yywrap()
{
	return 1;
}

int main(int argc,char* argv[])
{
	yyin=fopen(argv[1],"r");
	yylex();
	fclose(yyin);
}

Writing practical1.l


In [ ]:
%%writefile input.c 
#include<stdio.h>
void main()
{
int a,b;
char bilateral;
a=3;
c=a+d;
}

Writing input.c


In [ ]:
%%shell
lex practical1.l
gcc -o practical1 lex.yy.c
./practical1 input.c

#include<stdio.h>	==> PREPROCESSOR
void	==> DATATYPE
main	==> IDENTIFIER
(	==> PARANTHESIS BEGIN
)	==> PARENTHESIS END
{	==> BLOCK BEGIN
int	==> DATATYPE
a	==> IDENTIFIER
b	==> IDENTIFIER
;	==> DELIMITER
char	==> DATATYPE
bilateral	==> IDENTIFIER
;	==> DELIMITER
a	==> IDENTIFIER
=	==> ASSIGNMENT OP
3	==> CONSTAINT INTEGER
;	==> DELIMITER
c	==> IDENTIFIER
=	==> ASSIGNMENT OP
a	==> IDENTIFIER
+	==> ARITHMETIC OPERATOR
d	==> IDENTIFIER
;	==> DELIMITER
}	==> BLOCK END


# Practical 2
Implement a parser for an expression grammar using YACC and LEX for the subset of C. Cross check your output with Stanford LEX and YACC.
----

In [ ]:
%%writefile practical2.y
%{
#include<stdio.h>
int flag=0;
%}

%token NUMBER
%left '+' '-'
%left '*' '/' '%'
%left '(' ')'

%%

ArithmeticExpression: E{

         printf("\nResult=%d\n",$$);
         return 0;
        };

E:E'+'E {$$=$1+$3;}
 |E'-'E {$$=$1-$3;}
 |E'*'E {$$=$1*$3;}
 |E'/'E {$$=$1/$3;}
 |E'%'E {$$=$1%$3;}
 |'('E')' {$$=$2;}
 | NUMBER {$$=$1;}
;

%%
int main()
{
  yyparse();
  if(flag==0)
   printf("\nEntered arithmetic expression is Valid\n\n");
  return 0;
}

int yyerror()
{
   printf("\nEntered arithmetic expression is Invalid\n\n");
   flag=1;
   return 0;
}


Writing practical2.y


In [ ]:
%%writefile practical2.l
%{
#include<stdio.h>
#include "y.tab.h"
//extern int yylval;
%}

%%

[0-9]+ { yylval=atoi(yytext); return NUMBER; }

[\n] return 0;
. return yytext[0];

%%

int yywrap()
{
return 1;
}

Writing practical2.l


In [ ]:
%%shell
lex practical2.l
yacc -d practical2.y
gcc -o practical2 y.tab.c lex.yy.c -ll -ly -lm
./practical2
#5+55

y.tab.c: In function ‘yyparse’:
y.tab.c:1119:16: warning: implicit declaration of function ‘yylex’ [-Wimplicit-function-declaration]
       yychar = yylex ();
                ^~~~~
y.tab.c:1300:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? [-Wimplicit-function-declaration]
       yyerror (YY_("syntax error"));
       ^~~~~~~
       yyerrok
5+55

Result=60

Entered arithmetic expression is Valid



In [ ]:
%%shell
./practical2
#-5+5++8

5++5++8**-

Entered arithmetic expression is Invalid



# Practical 3
Generate and populate appropriate Symbol Table.
----

In [ ]:
%%writefile SYMTAB.java
import java.io.*;

public class SYMTAB{
    
  public static void main(String ar[])throws IOException{
      
    BufferedReader br=new BufferedReader(new InputStreamReader(System.in));

    int i;

    String a[][]={{"","START","101",""},
    {"","MOVER","BREG","ONE"},
    {"AGAIN","MULT","BREG","TERM"},
    {"","MOVER","CREG","TERM"},
    {"","ADD","CREG","N"},
    {"","MOVEM","CREG","TERM"},
    {"N","DS","2",""},
    {"RESULT","DS","2",""},
    {"ONE","DC","1",""},
    {"TERM","DS","1",""},
    {"","END","",""}};

    int lc=Integer.parseInt(a[0][2]);

    String st[][]=new String[5][2];

    int cnt=0,l;

    for (i=1;i<11;i++){
      if (a[i][0]!=""){
        st[cnt][0]=a[i][0];
        st[cnt][1]=Integer.toString(lc);
        cnt++;
          if(a[i][1]=="DS"){
            int d=Integer.parseInt(a[i][2]);
            lc=lc+d;
          }else{
            lc++;
          }
      }else{
        lc++;
      }
    }
  System.out.print("***SYMBOL TABLE****\n");

  System.out.println("_____________________");

  for(i=0;i<5;i++){
    for(cnt=0;cnt<2;cnt++){
      System.out.print(st[i][cnt]+"\t");
    }
    System.out.println();
  }

  }
}

Writing SYMTAB.java


In [ ]:
%%shell
javac SYMTAB.java
java SYMTAB

***SYMBOL TABLE****
_____________________
AGAIN	102	
N	106	
RESULT	108	
ONE	110	
TERM	111	


# Practical 4
Implementation of Semantic Analysis Operations (like type checking, verification of function parameters, variable declarations and coercions) possibly using an Attributed Translation Grammar.
----

In [ ]:
%%writefile practical4.l
%{
#include<stdio.h>
void yyerror(char *);
#include "y.tab.h"
%}

%%
void|int|float|char return BUILTIN;
, return COMMA;
; return SC;
"\n" return 0;
")" return RP;
"(" return LP;
[a-zA-Z][a-zA-Z0-9]* return ID;
%%

int yywrap(void) {
return 1;
}

Writing practical4.l


In [ ]:
%%writefile practical4.y
%{
#include<stdio.h>
int yylex(void);
void yyerror(char *);
int flag=0;
%}

%token COMMA ID BUILTIN SC RP LP

%%
var:BUILTIN EXP SC {flag=1;}
;
EXP:EXP COMMA ID| ID |ID LP BUILTIN ID RP {flag =1;}
;
%%


int main()
{ 

yyparse(); 
if(flag==1)
{
printf("valid Declaration\n");
}
else
{ printf("Invalid Declaration\n");
}
return 0;
}


void yyerror(char *s) 
{
printf(" ");
}

Writing practical4.y


In [ ]:
%%shell
lex practical4.l
yacc -d practical4.y
gcc -o practical4 y.tab.c lex.yy.c -ll -ly -lm
./practical4
#int a;

int a;
 valid Declaration


In [ ]:
%%shell
./practical4
#int ;

int ;
  Invalid Declaration


# Practical 5
Implement the front end of a compiler that generates the three address code for a simple language
----

In [ ]:
%%writefile practical5.l
ALPHA [A-Za-z]
DIGIT [0-9]
%{
 #include"y.tab.h"
%}

%%
{ALPHA}({ALPHA}|{DIGIT})* return ID;
{DIGIT}+ {yylval=atoi(yytext);return NUM;}
[\n\t] yyterminate();
. return yytext[0];
%%
int yywrap()
{
return 1;
}

/*

(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment5$ ./a.out
enter the expression:a=b+c*d
t0=c * d 
t1=b + t0 
a=t1
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment5$ 

*/

Writing practical5.l


In [ ]:
%%writefile practical5.y
%{
#include<string.h>
#include<stdio.h>
#include<ctype.h>
%}

%token ID NUM
%right '='
%left '+' '-'
%left '*' '/'
%left UMINUS

%%
S:ID{push();} '='{push();} E{codegen_assign();}
;
E:E '+'{push();} T{codegen();}
|E '-'{push();} T{codegen();}
|T
;
T:T '*'{push();} F{codegen();}
|T '/'{push();} F{codegen();}
|F
;
F: '(' E ')'
|'-'{push();} F{codegen_umin();} %prec UMINUS
|ID{push();}
|NUM{push();}
;
%%
#include"lex.yy.c"
char st[100][10];
int top=0;
char i_[2]="0";
char temp[2]="t";

int main()
{
printf("enter the expression:");
yyparse();
return 0;
}
push()
{
strcpy(st[++top],yytext);
}
int codegen()
{
strcpy(temp,"t");
strcat(temp,i_);
printf("%s=%s %s %s \n",temp,st[top-2],st[top-1],st[top]);
top-=2;
strcpy(st[top],temp);
i_[0]++;
return 0;
}

int codegen_umin()
{
strcpy(temp,"t");
strcat(temp,i_);
printf("%s=-%s\n",temp,st[top]);
top--;
strcpy(st[top],temp);
i_[0]++;
return 0;
}
int codegen_assign()
{
printf("%s=%s\n",st[top-2],st[top]);
top-=2;
return 0;
}

int yyerror(char *s)
{
printf("%s\n",s);
return 0;
}

Writing practical5.y


In [ ]:
%%shell
# lex practical5.l
# yacc -d practical5.y
# gcc lex.yy.c y.tab.c

----

In [ ]:
%%writefile practical_5.l
%{
 #include<stdio.h>
 #include"y.tab.h"
 char ch='a';

%}
%%
[0-9]+ {yylval.dval=yytext[0]; return NUM;} /*this would store value of integer
 in yyval;nd return token integer to yacc*/
\n {return 0;}
. {return yytext[0];}
%%
char intgencode(char ch1,char first,char op,char second)
{
 printf("\n%c = %c %c %c\n",ch,first,op,second);
 return ch++;
}
void yyerror(char* str)
{
 printf("\n%s",str);
}

Writing practical_5.l


In [ ]:
%%writefile practical_5.yacc
%{
#include<stdio.h>
void yyerror(char*);
int yylex();
int yywrap();
%}
%union
{
char dval; /*to define possible symbol types,is to allow
 storing different kind of objects into node emitted by lex*/
}
%token <dval> NUM
%type <dval> E
%left '+' '-'
%left '*' '/'
%%
statement : E {printf("\n t= %c\n This is Three Address Code ...\n",$1);}
E : E '+' E {$$=intgencode($$,$1,'+',$3);}
 | E '-' E {$$=intgencode($$,$1,'-',$3);}
 | E '*' E {$$=intgencode($$,$1,'*',$3);}
 | E '/' E {$$=intgencode($$,$1,'/',$3);}
 | '(' E ')' {$$=$2;}
 | NUM {$$=$1;}
 ;
%%
int main()
{
 yyparse();
 return 0;
}
int yywrap(){
return 1;
}

Writing practical_5.yacc


In [ ]:
%%shell
# lex practical_5.l
# yacc -d practical_5.yacc
# gcc -o practical_5 lex.yy.c y.tab.c  -ll -ly -lm
# ./practical_5


In [ ]:
%%writefile ThreeAddressCode.java
/*
	Program to implement Three Address Code in Java

*/
import java.io.*;

class ThreeAddressCode
{
	private static final char[][] precedence = {
		{'/', '1'},
		{'*', '1'},
		{'+', '2'},
		{'-', '2'}
	};
	
	private static int precedenceOf(String t)
	{
		char token = t.charAt(0);
		for (int i=0; i < precedence.length; i++)
		{
			if (token == precedence[i][0])
			{
				return Integer.parseInt(precedence[i][1]+"");
			}
		}
		return -1;
	}
	
	public static void main(String[] args) throws Exception
	{
		int i, j, opc=0;
		char token;
		boolean processed[];
		String[][] operators = new String[10][2];
		String expr="", temp;
		BufferedReader in = new BufferedReader(new InputStreamReader(System.in));
		System.out.print("\nEnter an expression: ");
		expr = in.readLine();
		processed = new boolean[expr.length()];
		for (i=0; i < processed.length; i++)
		{
			processed[i] = false;
		}
		for (i=0; i < expr.length(); i++)
		{
			token = expr.charAt(i);
			for (j=0; j < precedence.length; j++)
			{
				if (token==precedence[j][0])
				{
					operators[opc][0] = token+"";
					operators[opc][1] = i+"";
					opc++;
					break;
				}
			}
		}
		System.out.println("\nOperators:\nOperator\tLocation");
		for (i=0; i < opc; i++)
		{
			System.out.println(operators[i][0] + "\t\t" + operators[i][1]);
		}
		//sort
		for (i=opc-1; i >= 0; i--)
		{
			for (j=0; j < i; j++)
			{
				if (precedenceOf(operators[j][0]) > precedenceOf(operators[j+1][0]))
				{
					temp = operators[j][0];
					operators[j][0] = operators[j+1][0];
					operators[j+1][0] = temp;
					temp = operators[j][1];
					operators[j][1] = operators[j+1][1];
					operators[j+1][1] = temp;
				}				
			}
		}
		System.out.println("\nOperators sorted in their precedence:\nOperator\tLocation");
		for (i=0; i < opc; i++)
		{
			System.out.println(operators[i][0] + "\t\t" + operators[i][1]);
		}
		System.out.println();
		for (i=0; i < opc; i++)
		{
			j = Integer.parseInt(operators[i][1]+"");
			String op1="", op2="";
			if (processed[j-1]==true)
			{
				if (precedenceOf(operators[i-1][0]) == precedenceOf(operators[i][0]))
				{
					op1 = "t"+i;
				}
				else
				{
					for (int x=0; x < opc; x++)
					{
						if ((j-2) == Integer.parseInt(operators[x][1]))
						{
							op1 = "t"+(x+1)+"";
						}
					}
				}
			}
			else
			{
				op1 = expr.charAt(j-1)+"";
			}
			if (processed[j+1]==true)
			{
				for (int x=0; x < opc; x++)
				{
					if ((j+2) == Integer.parseInt(operators[x][1]))
					{
						op2 = "t"+(x+1)+"";
					}
				}
			}
			else
			{
				op2 = expr.charAt(j+1)+"";
			}
			System.out.println("t"+(i+1)+" = "+op1+operators[i][0]+op2);
			processed[j] = processed[j-1] = processed[j+1] = true;
		}
	}
}

Writing ThreeAddressCode.java


In [ ]:
%%shell 
javac ThreeAddressCode.java
java ThreeAddressCode
# a*b/c+d-e*f


Enter an expression: a*b/c+d-e*f

Operators:
Operator	Location
*		1
/		3
+		5
-		7
*		9

Operators sorted in their precedence:
Operator	Location
*		1
/		3
*		9
+		5
-		7

t1 = a*b
t2 = t1/c
t3 = e*f
t4 = t2+d
t5 = t4-t3


# Practical 6
A Register Allocation algorithm that translates the given code into ne with a fixed number of registers.
----

In [ ]:
%%writefile input.c
t1=c*d
t2=b+t1
t3=e /f
t4=t2-t3
a=t4

Overwriting input.c


In [ ]:
%%writefile practical6.l
%{
#include <stdio.h>
#include "y.tab.h"
%}
%%
[a-z][a-zA-Z0-9]*|[0-9]+ { strcpy(yylval.vname,yytext); return NAME; }
[ |\t] ;
.|\n { return yytext[0]; }
%%

Writing practical6.l


In [ ]:
%%writefile practical6.y
%{
	#include<stdio.h>
	#include<ctype.h>
	#include<string.h>
	FILE *fpOut;
	int yylex();
	int yyerror();
	int fcloseall();
%}
%union
{

	char vname[10];
	int val;

}
%left '+' '-'
%left '*' '/'
%token <vname> NAME
%type <vname> expression
%%
input : line '\n' input
		| '\n' input
		| ;
line : NAME '=' expression { fprintf(fpOut,"MOV %s, AX\n",$1); }
;
expression: NAME '+' NAME
	{
	fprintf(fpOut,"MOV AX, %s\n",$1);
	fprintf(fpOut,"ADD AX, %s\n",$3);
	}
	| NAME '-' NAME
	{
	fprintf(fpOut,"MOV AX, %s\n",$1);
	fprintf(fpOut,"SUB AX, %s\n",$3);
	}
	| NAME '*' NAME
	{
	fprintf(fpOut,"MOV AX, %s\n",$1);
	fprintf(fpOut,"MUL AX, %s\n",$3);
	}
	| NAME '/' NAME
	{
	fprintf(fpOut,"MOV AX, %s\n",$1);
	fprintf(fpOut,"DIV AX, %s\n",$3);
	}| NAME
	{
	fprintf(fpOut,"MOV AX, %s\n",$1);
	strcpy($$, $1);
	}
;
%%
FILE *yyin;
int main()
{
	FILE *fpInput;
	fpInput = fopen("input.c","r");
	if(fpInput=='\0')
	{
	printf("file read error");
	//exit(0);
	}
	fpOut = fopen("out.c","w");
	if(fpOut=='\0')
	{
	printf("file creation error");
	//exit(0);
	}
	yyin = fpInput;
	yyparse();
	fcloseall();
}
int yyerror(char *msg)
{
	printf("%s",msg);
}
/* OUTPUT-
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment6$ lex assign6.l
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment6$ yacc -d assign6.y
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment6$ gcc y.tab.c lex.yy.c -ll
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment6$ ./a.out
NAME token is t1 
token is = 
NAME token is c 
token is * 
NAME token is d 
token is 
 
NAME token is t2 
token is = 
NAME token is b 
token is + 
NAME token is t1 
token is 
 
NAME token is t3 
token is = 
NAME token is e 
token is / 
NAME token is f 
token is 
 
NAME token is t4 
token is = 
NAME token is t2 
token is + 
NAME token is t3 
token is 
 
NAME token is a 
token is = 
NAME token is t4 
token is 
 
(base) dell@dell-Inspiron-15-3567:~/BEALL2/Compiler/Compiler/Assignment6$ 
*/


Writing practical6.y


In [ ]:
%%shell
lex practical6.l
yacc -d practical6.y
gcc y.tab.c lex.yy.c -ll
# ./a.out

practical6.y: In function ‘main’:
practical6.y:58:12: warning: comparison between pointer and zero character constant [-Wpointer-compare]
  if(fpInput=='\0')
            ^~
practical6.y:58:5: note: did you mean to dereference the pointer?
  if(fpInput=='\0')
     ^
practical6.y:64:10: warning: comparison between pointer and zero character constant [-Wpointer-compare]
  if(fpOut=='\0')
          ^~
practical6.y:64:5: note: did you mean to dereference the pointer?
  if(fpOut=='\0')
     ^


# Practical 7

Implementation of Instruction Scheduling Algorithm. 
----

In [ ]:
%%writefile FCFS.java
import java.util.Scanner;

public class FCFS {
  int Count=0;
  int[][] processData;
  Scanner scan = new Scanner(System.in);

  public void scheduleFCFS(){
    processData[0][2] = 0;
    processData[0][3] = processData[0][1];
    for(int i=1;i<Count;i++){
      processData[i][2] = processData[i-1][3];
      processData[i][3] = processData[i][1]+processData[i][2];
    }
  }

  public void setValue(){
    System.out.print("Enter Number Of processes : ");
    Count = scan.nextInt();
    processData = new int[Count][4];
    for(int i=0;i<Count;i++){
      System.out.print("Processes["+i+"] :-> ");
      processData[i][0] = i;
      processData[i][1] = scan.nextInt();
    }
  }
  public void DisplayValue(){
    System.out.println("------------------------------------------------------------------------");
    System.out.println("Process Number || Brust Time || WatingTime || Turn Around Time");
    System.out.println("------------------------------------------------------------------------");
    for(int i=0;i<Count;i++){
      System.out.println("Processes["+processData[i][0]+"]\t||\t"+processData[i][1]+"\t||\t"+processData[i][2]+"\t||\t"+processData[i][3]);
    }
    System.out.println("------------------------------------------------------------------------");
  }

  public static void main(String a[]){
    FCFS fs = new FCFS();
    fs.setValue();
    System.out.println("\n\n***********First Come First Server :(FCFS) :***********\n");
    System.out.println("\n\nBefore Scheduling\n");
    fs.DisplayValue();
    fs.scheduleFCFS();
    System.out.println("\n\nAfter Scheduling\n");
    fs.DisplayValue();
  }
}

Writing FCFS.java


In [ ]:
%%shell
javac FCFS.java
java FCFS
# 7

# 1
# 9
# 5
# 3
# 2
# 2
# 7

Enter Number Of processes : 7
Processes[0] :-> 1
Processes[1] :-> 9
Processes[2] :-> 5
Processes[3] :-> 3
Processes[4] :-> 2
Processes[5] :-> 2
Processes[6] :-> 7


***********First Come First Server :(FCFS) :***********



Before Scheduling

------------------------------------------------------------------------
Process Number || Brust Time || WatingTime || Turn Around Time
------------------------------------------------------------------------
Processes[0]	||	1	||	0	||	0
Processes[1]	||	9	||	0	||	0
Processes[2]	||	5	||	0	||	0
Processes[3]	||	3	||	0	||	0
Processes[4]	||	2	||	0	||	0
Processes[5]	||	2	||	0	||	0
Processes[6]	||	7	||	0	||	0
------------------------------------------------------------------------


After Scheduling

------------------------------------------------------------------------
Process Number || Brust Time || WatingTime || Turn Around Time
------------------------------------------------------------------------
Processes[0]	||	1	||	0	||	1
Processes[1]	||	9	||	1

In [ ]:
%%writefile RoundRobin.java
import java.util.Scanner;
public class RoundRobin {
  int Count=0;
  int[][] processData;

  int[] CCycle; // Complete Cycle
  int[] PCycle; // Partial Cycle
  int[] TCycle; // Total Cycle

  String[] Queue;
  boolean[] Status;
  int[] startQue;
  int[] endQue;

  int RTT = 0;
  Scanner scan = new Scanner(System.in);

  public void scheduleRR(){
    int TotalProcess = CCycle.length;

    for(int i = 0,j=0;i < Queue.length || j<Queue.length;i++){
        
      int ProNo = i % TotalProcess;
      if(CCycle[ProNo] != 0){
        Queue[j] = "process["+ProNo+"]";
        if(j==0)
        startQue[j] = 0;
        else
        startQue[j] = endQue[j-1] + 1;
        if(!Status[ProNo]){
          processData[ProNo][2] = startQue[j];
          Status[ProNo] = true;
        }
        endQue[j] = startQue[j]+RTT-1;
        j++;
        CCycle[ProNo] = CCycle[ProNo] - 1;
      }else{
        if(Status[ProNo]){
          Queue[j] = "process["+ProNo+"]";
          if(j==0)
          startQue[j] = 0;
          else
          startQue[j] = endQue[j-1] + 1;
          endQue[j] = startQue[j]+PCycle[ProNo]-1;
          processData[ProNo][3] = endQue[j];
          PCycle[ProNo]=0;
          j++;
          Status[ProNo]= false;
        }
      }
    }

    System.out.println("Queue : start : End : Interval : ");

    for (int i = 0; i < Queue.length ; i++){
      int temp = (endQue[i]-startQue[i]+1);
      System.out.println(" "+Queue[i]+" "+startQue[i]+" "+endQue[i] +""+temp );
      if(i%Count == Count-1)
      System.out.println("");
    }
  }

  public void setValue(){
    System.out.print("Enter Number Of processes : ");
    Count = scan.nextInt();

    System.out.print("Enter Round Trip Time : ");
    RTT = scan.nextInt();

    processData = new int[Count][4];
    CCycle = new int[Count];
    PCycle = new int[Count];
    TCycle = new int[Count];

    for(int i=0;i<Count;i++){
      System.out.print("Processes["+i+"] :-> ");
      processData[i][0] = i;
      processData[i][1] = scan.nextInt();
      CCycle[i] = processData[i][1]/RTT;
      PCycle[i] = processData[i][1]%RTT;
      if(PCycle[i]==0)
      TCycle[i] = CCycle[i];
      else
      TCycle[i] = CCycle[i] + 1;
      System.out.println("P["+i+"] CC : "+CCycle[i]+" PC : "+PCycle[i]+" TC :"+TCycle[i]);
    }

    Status = new boolean[Count];
    for(int i=0;i<Count;i++){
      Status[i]= false;
    }
    initQueue();
  }

  public void initQueue(){
    int MaxQueLen = 0;
    for(int i=0;i<TCycle.length;i++){
      MaxQueLen = MaxQueLen + TCycle[i];
    }
    Queue = new String[MaxQueLen];
    startQue = new int[MaxQueLen];
    endQue = new int[MaxQueLen];
    System.out.println("Queue Length : "+MaxQueLen);
  }

  public void DisplayValue(){
    System.out.println("------------------------------------------------------------------------");
    System.out.println("Process Number || Brust Time || Wating Time || TurnAround Time");
    System.out.println("------------------------------------------------------------------------");
    for(int i=0;i<Count;i++){
      System.out.println("Processes["+processData[i]
      [0]+"]\t||\t"+processData[i][1]+"\t||\t"+processData[i][2]+"\t||\t"+processData[i][3]);
    }
    System.out.println("-------------------------------------------------------------------------");
  }
  
  public static void main(String a[]){
    RoundRobin rr = new RoundRobin();
    rr.setValue();
    System.out.println("\n\n***********Round Robin : (RR) :***********\n");
    System.out.println("\n\nBefore Scheduling\n");
    rr.DisplayValue();
    rr.scheduleRR();
    System.out.println("\n\nAfter Scheduling\n");
    rr.DisplayValue();
  }
}

Writing RoundRobin.java


In [ ]:
%%shell
javac RoundRobin.java
java RoundRobin

# 7
# 3

# 2
# 5
# 3
# 9
# 1
# 6
# 1

Enter Number Of processes : 7
Enter Round Trip Time : 3
Processes[0] :-> 2
P[0] CC : 0 PC : 2 TC :1
Processes[1] :-> 5
P[1] CC : 1 PC : 2 TC :2
Processes[2] :-> 3
P[2] CC : 1 PC : 0 TC :1
Processes[3] :-> 9
P[3] CC : 3 PC : 0 TC :3
Processes[4] :-> 1
P[4] CC : 0 PC : 1 TC :1
Processes[5] :-> 6
P[5] CC : 2 PC : 0 TC :2
Processes[6] :-> 1
P[6] CC : 0 PC : 1 TC :1
Queue Length : 11


***********Round Robin : (RR) :***********



Before Scheduling

------------------------------------------------------------------------
Process Number || Brust Time || Wating Time || TurnAround Time
------------------------------------------------------------------------
Processes[0]	||	2	||	0	||	0
Processes[1]	||	5	||	0	||	0
Processes[2]	||	3	||	0	||	0
Processes[3]	||	9	||	0	||	0
Processes[4]	||	1	||	0	||	0
Processes[5]	||	6	||	0	||	0
Processes[6]	||	1	||	0	||	0
-------------------------------------------------------------------------
Queue : start : End : Interval : 
 process[1] 0 23
 process[2] 3 53
 pro

In [ ]:
%%writefile SJF.java
import java.util.Scanner;
public class SJF {
  int Count=0;
  int[][] processData;
  Scanner scan = new Scanner(System.in);

  public static void main(String a[]){
    System.out.println("\n\n***********Shortest Job First : (SJF) :***********\n");
    SJF sjf = new SJF();
    sjf.setValue();
    System.out.println("\n\nBefore Scheduling\n");
    sjf.DisplayValue();
    sjf.scheduleSJF();
    System.out.println("\n\nAfter Scheduling\n");
    sjf.DisplayValue();
  }

  public void scheduleSJF(){
    scheduleSorting();
    scheduleCalculation();
  }

  public void scheduleSorting(){
    for(int i=0;i<Count;i++){
      int ele=i;
      int eleData=processData[i][1];
      for(int j=i+1;j<Count;j++){
        if(eleData > processData[j][1]){
          eleData= processData[j][1];
          ele=processData[j][0];
        }
      }
      swapProcess(i,ele);
    }
  }

  public void swapProcess(int p1,int p2){
    int[] temp = processData[p1];
    processData[p1] = processData[p2];
    processData[p2] = temp;
  }

  public void scheduleCalculation(){
    processData[0][2] = 0;
    processData[0][3] = processData[0][1];
    for(int i=1;i<Count;i++){
      processData[i][2] = processData[i-1][3];
      processData[i][3] = processData[i][1]+processData[i][2];
    }
  }

  public void setValue(){
    System.out.print("Enter Number Of processes : ");
    Count = scan.nextInt();
    processData = new int[Count][4];
    for(int i=0;i<Count;i++){
      System.out.println("Processes["+i+"] :-> ");
      processData[i][0] = i;
      System.out.print("\t\tBrust Time["+i+"] :-> ");
      processData[i][1] = scan.nextInt();
    }
  }

  public void DisplayValue(){
    System.out.println("------------------------------------------------------------------------");
    System.out.println("Process Number || Brust Time || Wating Time || TurnAround Time");
    System.out.println("------------------------------------------------------------------------");
    for(int i=0;i<Count;i++){
      System.out.println("Processes["+processData[i]
      [0]+"]\t||\t"+processData[i][1]+"\t||\t"+processData[i][2]+"\t||\t"+processData[i][3]);
    }
    System.out.println("-------------------------------------------------------------------------");
  }
}

Writing SJF.java


In [ ]:
%%shell
javac SJF.java
java SJF

# 7

# 2
# 5
# 3
# 9
# 1
# 6
# 1



***********Shortest Job First : (SJF) :***********

Enter Number Of processes : 7
Processes[0] :-> 
		Brust Time[0] :-> 2
Processes[1] :-> 
		Brust Time[1] :-> 5
Processes[2] :-> 
		Brust Time[2] :-> 3
Processes[3] :-> 
		Brust Time[3] :-> 9
Processes[4] :-> 
		Brust Time[4] :-> 1
Processes[5] :-> 
		Brust Time[5] :-> 6
Processes[6] :-> 
		Brust Time[6] :-> 1


Before Scheduling

------------------------------------------------------------------------
Process Number || Brust Time || Wating Time || TurnAround Time
------------------------------------------------------------------------
Processes[0]	||	2	||	0	||	0
Processes[1]	||	5	||	0	||	0
Processes[2]	||	3	||	0	||	0
Processes[3]	||	9	||	0	||	0
Processes[4]	||	1	||	0	||	0
Processes[5]	||	6	||	0	||	0
Processes[6]	||	1	||	0	||	0
-------------------------------------------------------------------------


After Scheduling

------------------------------------------------------------------------
Process Number || Brust Time || Wating Time